# 擴增資料集

說明：
擴增有標籤之影像，擴增方法為隨機水平翻轉、隨機salt & pepper noise、隨機gaussian noise

### Inputs
- `ROOT`     - 輸入影像/標籤根目錄
- `saveROOT` - 儲存影像/標籤根目錄

組合示意如下：
```
+-------+-------+-------+  
|       |   -   |h-flip |  
+-------+-------+-------+  
|none   |   -   |    na |  
+-------+-------+-------+  
|s&p    |    na |    na |  
+-------+-------+-------+  
|gauss  |    na |    na |  
+-------+-------+-------+  
|s&p+g  |    na |    na |  
+-------+-------+-------+  
```

根目錄資料夾樹狀圖:

- **注意**：影像與標籤要在同一個資料夾
- 資料夾可以分好幾層，不一定只能一層而已 

```
ROOT  
└─ image_and_label_folder  
   ├─ image01.jpg  
   ├─ image01.txt  
   ├─ ..  
   ├─ imageXX.jpg  
   └─ imageXX.txt  
 
```

輸出根目錄資料夾樹狀圖：
```
saveROOT  
├─ image_and_label_folder  
│  ├─ image01.jpg  
│  ├─ image01.txt  
│  ├─ ..  
│  ├─ imageXX.jpg  
│  └─ imageXX.txt  
└─ image_augmentation_log.txt       # 程式產生
 
```

In [1]:
import os
import cv2
import random
import numpy as np
from tqdm.notebook import tqdm # jupyter notebook的進度條

In [2]:
# VARIABLE
ROOT = "D:\\Dataset\\Sumitomo(CM088A)\\220316\\"
saveROOT = "D:\\Dataset\\Sumitomo(CM088A)\\220316_augment\\"

In [3]:
## @brief Description: 建立資料夾
#  @param folder
#  
#  @return None
#  @date 20220321  danielwu
def MakeDirs(folder):
    if not os.path.isdir(folder):
        os.makedirs(folder)

In [4]:
## @brief Description: 取得母資料夾內所有特定格式檔案路徑
#  @param [in] folder     母資料夾路徑
#  @param [in] extension  副檔名
#  
#  @return paths
#  @date 20220321  danielwu
def GetPaths(folder, extension='.MOV'):
    paths = []

    for response in os.walk(folder):        # response = (dirpath, dirname, filenames)
        if response[2] != []:               # look for filenames
            for f in response[2]:
                if f.endswith(extension):  # only append .MOV file in paths 
                    paths.append(os.path.join(response[0], f))
    
    return paths

In [5]:
## @brief Description: 從YOLO標籤的文件檔取得bbox資訊
#  @param [in] txt_path    YOLO標籤的文件檔路徑
#  
#  @return info      標籤bbox資訊
#  @date 20220610  danielwu
def TxtInfo(txt_path):
    info = []
    with open(txt_path, 'r') as f:
        for line in f.readlines():
            line = line.split('\n')[0]
            if line != '':
                info.append(line)
    
    return info

In [6]:
## @brief Description: 選擇(1) 是否水平翻轉 (2) 是否模糊處理
#  @param [in] channel 水平翻轉feat三種模糊處理，扣掉都不處理後共7種
#  
#  @return flip, noisy      
#  @date 20220610  danielwu
def Mode(channel):
    flip = False
    noisy = "none"
    if channel % 2:
        flip = True
    if channel in [2, 3]:
        noisy = "s&p"
    if channel in [4, 5]:
        noisy = "gauss"
    if channel in [6, 7]:
        noisy = "s&p gauss"
    
    return flip, noisy

In [7]:
## @brief Description: 影像水平翻轉後的bbox資訊
#  @param [in] bbox    bbox資訊(conf, x, y, w, h)
#  
#  @return new_bbox    翻轉後的bbox
#  @date 20220610  danielwu
def FlipHorizontalAnnotation(bbox):
    conf, x, y, w, h = bbox.split(" ")
    flip_x = "%.3f" %round(1 - float(x), 3)
    
    return "{} {} {} {} {}".format(conf, flip_x, y, w, h)

In [8]:
## @brief Description: 影像模糊化 (高斯模糊或黑白噪點)
#  @param [in] noise_type    模糊效果 (Gaussian noise or Salt and pepper noise)
#  @param [in] image         輸入影像   
#  
#  @return output  輸出模糊影像
#  @date 20220610  danielwu
def Noisy(noise_type, image):
    if noise_type == "gauss":
        '''
        Add gauissian noise to image
        ksize: Gaussian kernel size. ksize.width and ksize.height can differ but they both must be positive and odd.
        '''
        ksize = (random.randrange(1,13,2), random.randrange(1,13,2))
        return cv2.GaussianBlur(image, ksize, 0)
    elif noise_type == "s&p":
        '''
        Add salt and pepper noise to image
        prob: Probability of the noise
        '''
        output = np.zeros(image.shape,np.uint8)
        prob = round(random.uniform(0.01, 0.1), 3)
        thres = 1 - prob 
        for i in range(image.shape[0]):
            for j in range(image.shape[1]):
                rdn = random.random()
                if rdn < prob:
                    output[i][j] = 0
                elif rdn > thres:
                    output[i][j] = 255
                else:
                    output[i][j] = image[i][j]
        return output
    else:
        '''
        Not add any noise to image
        '''
        return image

In [9]:
if __name__ == "__main__":
    # Create save root folder
    MakeDirs(saveROOT)
    # Set log fiel path and open it
    log_path = os.path.join(saveROOT, "image_augmentation_log.txt")
    log = open(log_path, "w")
    counts = np.zeros((7,), dtype=int)
    # Get the annotation file paths
    txt_paths = GetPaths(ROOT, extension=".txt")
    
    for txt_path in tqdm(txt_paths):
        ori_annotation = TxtInfo(txt_path)
        image_path = txt_path.replace(".txt", ".jpg")
        
        # Check origin annotation is not empty and image is exist
        if ori_annotation and os.path.isfile(image_path):
            # Set save dir and create save dir if not exist
            image_dir, image_basename = os.path.split(image_path)
            relative_image_dir = image_dir.split(ROOT)[1]
            save_dir = os.path.join(saveROOT, relative_image_dir)
            MakeDirs(save_dir)            
            # Set save txt path and save image path
            save_txt = os.path.join(save_dir, image_basename.replace(".jpg", ".txt"))
            save_image = os.path.join(save_dir, image_basename)
            
            # Set flip or not and noisy mode (salt and pepper or gaussian or both)
            channel = random.randint(1, 7)
            flip, noisy = Mode(channel)
            
            # Create the new annotation file, convert x if flip is true and write down the info
            with open(save_txt, "w") as f:
                for info in ori_annotation:
                    if flip:
                        info = FlipHorizontalAnnotation(info)
                    if info != ori_annotation[-1]:
                        f.write("{}\n".format(info))
                    else:
                        f.write(info)
            
            # Create and save the new image, flip horizontal if flip is true
            image = cv2.imread(image_path)
            if flip:
                image = cv2.flip(image, 1)
            # Add noise according noisy value
            if noisy == "s&p gauss":
                noise_image = Noisy("s&p", image)
                noise_image = Noisy("gauss", noise_image)
            else:
                noise_image = Noisy(noisy, image)
            cv2.imwrite(save_image, noise_image)
            
            # Save log
            log.write(save_image + "\n")
            log.write("  ├─ flip:  {}\n".format(flip))
            log.write("  └─ noisy: {}\n".format(noisy))
            counts[channel - 1] += 1
    

    # Save log (distributed)
    index = 1
    log.write("\n+-------+-------+-------+\n")
    for row in ["", "none", "s&p", "gauss", "s&p+g"]:
        if not row:
            log.write("|\t|   -\t|h-flip\t|\n")
        elif row == "none":
            log.write("|{}\t|   -\t|{:6d}\t|\n".format(row, counts[0]))
        else:
            log.write("|{}\t|{:6d}\t|{:6d}\t|\n".format(row, counts[index], counts[index + 1]))
            index += 2
        log.write("+-------+-------+-------+\n")
    log.close()

  0%|          | 0/69199 [00:00<?, ?it/s]